# libs

In [1]:
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim
from scipy import spatial
import glob
import os
import pandas as pd
import requests

# Data

## df pararius

In [2]:
list_of_files = glob.glob("data/temp/df_pararius_*.csv") # * means all if need specific format then *.csv
latest_file = max(list_of_files, key=os.path.getctime)
df_pararius = pd.read_csv(latest_file, index_col=[0])

## zipcodeZ

In [3]:
df_zipcode = pd.read_csv('data/processed/zipcodeZ.csv',index_col=[0])

# What is new?
These are new postcodes that we do not have in our database

In [4]:
temp1 = df_pararius['postcode'].unique()
temp2 = df_zipcode['postcode'].unique()
cep_zip = list(set(temp1) - set(temp2))
print(len(cep_zip))

361


## crawler
get data from "postcode bij adres"

In [5]:
def html_content(cep):
    url = 'https://postcodebijadres.nl/' + cep
    headers = {"User-Agent": "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:88.0) Gecko/20100101 Firefox/88.0"}
    soup = BeautifulSoup(requests.get(url, headers=headers).content, "html.parser")
    return soup

In [6]:
zipcode=pd.DataFrame({})
description_1={}

for cep in cep_zip:
    soup = html_content(cep)
    
    try:
        
        for num in range(9):
            
            split0 = soup.find_all('table',{"class":"table table-bordered mt-3"})[0]
            split1 = soup.find_all('table',{"class":"table table-bordered mt-3"})[1]
            
            split0_key = split0.find_all('th')[num].text
            split0_val = split0.find_all('td')[num].text

            lat = split1.find_all('td')[1].text
            lon = split1.find_all('td')[3].text
                                   
            description_1.update({
                split0_key:split0_val,
                'latitude':lat,
                'longitude':lon
            })


        zipcode_ = pd.DataFrame(description_1,index=[0])
        zipcode  = pd.concat([zipcode,zipcode_],0)
        
        
        zipcode.to_csv(f'data/temp/cep/zipcode_{cep}.csv')
        print(cep)
    except:
        
        print('#####################################',cep)
        pass

<ipython-input-6-9418347b4fe1>:28: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  zipcode  = pd.concat([zipcode,zipcode_],0)


2555VH
1749BJ
5261GD
1053PZ
3811ML
1073XB
3043AH
4871CK
5038KG
3527JH
3581HG
1321GA
1058DC
3083KL
1059BV
5682JP
1083TT
2555TL
9718AS
3555ER
2562TX
1261ZW
3742AA
6821CJ
7411CW
1087NE
1403RH
1181AL
7514AK
2561TR
9711NA
2514KE
2518GT
##################################### 5617BH
8111ND
1404DN
2596XZ
3511AD
2565KH
2596EP
9724BC
1622EG
2042LA
6826KV
5623AX
2011AS
3122HK
2629HM
3562AV
1053BJ
1421AC
7316GH
1404GZ
1012CK
##################################### 3431HM
1021GN
1541WP
3513VM
6711HA
1394GR
1075TV
1051JJ
3067JL
1211BR
1072BV
5025DP
2171KN
3022BB
1405GW
2011KX
1016DJ
##################################### 6219NM
3582JB
6224XD
5554AD
6828HV
1073KZ
2225XN
1017EN
2522GM
1811GP
1186HC
1012AJ
1106DS
3073ZT
5211NG
2719TJ
2555EP
5211TW
5021AN
1336LJ
6814GG
3039ZK
3024RS
3572BC
5709DK
1187ED
5612NE
7421GS
3073HJ
##################################### 1384AS
##################################### 9861DE
3207MS
2563BA
3431LE
8507CL
1058ET
8916BN
6538TJ
1324HH
2903AC
6543JL
1181HK
9746BV
2324HE
2552X

## cleaning data and save

In [7]:
try:
    zipcode['Straat'] = zipcode['Straat'].str.replace('\n',"",regex=True)
    zipcode['Postcode'] = zipcode['Postcode'].str.replace(' ',"",regex=True)
    zipcode = zipcode.rename(columns={'Postcode':'postcode'})
    zipcode = pd.concat([zipcode,df_zipcode],0).reset_index(drop=True)
    zipcode = zipcode.drop_duplicates(subset=['postcode']).reset_index(drop=True)
except:
    zipcode = df_zipcode.copy()

<ipython-input-7-ac7bc87cbddd>:5: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  zipcode = pd.concat([zipcode,df_zipcode],0).reset_index(drop=True)


In [8]:
zipcode_pararius_coo = pd.merge(df_pararius, zipcode, how='left',on='postcode')

# organizing before saving

In [9]:
zipcode_pararius_coo = zipcode_pararius_coo.fillna(0).drop_duplicates(subset=['irl']).reset_index(drop=True)

In [10]:
zipcode_pararius_coo.loc[zipcode_pararius_coo['interior']==0, 'interior'] = 'Upholstered'
zipcode_pararius_coo.loc[zipcode_pararius_coo['status']==0, 'status'] = 'Free'

In [11]:
# zipcode_pararius_coo['city'] = zipcode_pararius_coo['irl'].str.split("/",expand=True)[2].str.replace("-", " ").str.title()

In [12]:
# prepare link on streamlit
zipcode_pararius_coo['url'] = 'https://www.pararius.com' + zipcode_pararius_coo['irl']
zipcode_pararius_coo['link'] = "<a target='_blank' href=" + (zipcode_pararius_coo['url']).astype(str) + ">" + (zipcode_pararius_coo['street']).astype(str) + "</a>"
zipcode_pararius_coo['img'] = "<a href=" + zipcode_pararius_coo['url'] + " target='blank'><img src=" + zipcode_pararius_coo['image'] + " title='rent' width='150' height='100'/></a>"

# save data

In [13]:
zipcode_pararius_coo.to_csv('app/df_coo_pararius.csv')

In [14]:
zipcode.to_csv('data/processed/zipcodeZ.csv')